<a href="https://colab.research.google.com/github/midas-ruin/bqwvibu/blob/main/google-colab/Deep_Live_For_Colab_Refined(Working).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install InsightFace and Dependencies:**



 Run the following code to install the required packages.



 Note: This installation varies and depends on your cuda and cudnn version. Incase of face swapping issue refer to the link in the debug session to know the version to install.

In [1]:
#!pip install onnx==1.16.0
!pip install onnxruntime-gpu==1.20.1 #--extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
!pip install insightface #==0.7.3
#!pip install onnxruntime==1.18.0
!apt-get install -y ffmpeg
!ffmpeg -version
!mkdir deepfakecollab

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --ena

### DeepFakeLive

Follow the steps below to create the necessery directories for the project

In [2]:
!cd deepfakecollab
!mkdir deepfakecollab/Scripts

### [Optional] FRP



Follow the steps below to setup FRP. You will also need to host the FRPS on your VPS (free).



Note: You cannot run both FRP and ngrok together. Its ether FRP and ngrok

In [3]:

!touch deepfakecollab/Scripts/get_frs.sh
getfrs = """#!/usr/bin/env bash
# Check if frpc is installed
command -v frpc >/dev/null 2>&1
if [[ $? -ne 0 ]]; then
    echo "frpc is not found, installing..."
    wget -q -nc https://github.com/fatedier/frp/releases/download/v0.59.0/frp_0.59.0_linux_amd64.tar.gz
    tar -xzf frp_0.59.0_linux_amd64.tar.gz
    echo "Done!"
fi"""
with open('deepfakecollab/Scripts/get_frs.sh', 'w') as f:
    f.write(getfrs)
!touch deepfakecollab/Scripts/open_tunnel_frs.sh
getfrs = """#!/usr/bin/env bash
cmd="frp_0.59.0_linux_amd64/frpc -c frp_0.59.0_linux_amd64/frpc.toml"
kill -9 $(ps aux | grep $cmd | awk '{print $2}') 2> /dev/null
echo Opening tunnel
$cmd"""
with open('deepfakecollab/Scripts/open_tunnel_frs.sh', 'w') as f:
    f.write(getfrs)
!chmod +x deepfakecollab/Scripts/get_frs.sh
!chmod +x deepfakecollab/Scripts/open_tunnel_frs.sh
!deepfakecollab/Scripts/get_frs.sh

frpc is not found, installing...
Done!


### [Optional]  Ngrok



Follow to Setup Ngrok.



Note: You need an API key from Ngrok to use tcp for free you would need to add a billing details to their platform

In [4]:
!touch deepfakecollab/Scripts/get_ngrok.sh
getfrs = """#!/usr/bin/env bash
# Check if frpc is installed
command -v frpc >/dev/null 2>&1
if [[ $? -ne 0 ]]; then
    echo "ngrok is not found, installing..."
    wget -q -nc https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
    tar -xzf ngrok-v3-stable-linux-amd64.tgz
    echo "Done!"
fi"""
with open('deepfakecollab/Scripts/get_ngrok.sh', 'w') as f:
    f.write(getfrs)
!touch deepfakecollab/Scripts/open_tunnel_ngrok.sh
getfrs = """#!/usr/bin/env bash
cmd="./ngrok start --all --config ngrok.conf"
kill -9 $(ps aux | grep $cmd | awk '{print $2}') 2> /dev/null
echo Opening tunnel
$cmd"""
with open('deepfakecollab/Scripts/open_tunnel_ngrok.sh', 'w') as f:
    f.write(getfrs)
!chmod +x deepfakecollab/Scripts/get_ngrok.sh
!chmod +x deepfakecollab/Scripts/open_tunnel_ngrok.sh
!deepfakecollab/Scripts/get_ngrok.sh

ngrok is not found, installing...
Done!


### Create Model Folder



Create and download into the model folder

In [3]:
!mkdir -p deepfakecollab/Model
!wget https://huggingface.co/ninjawick/webui-faceswap-unlocked/resolve/main/inswapper_128.onnx -P /content/deepfakecollab/Model
!wget https://huggingface.co/hacksider/deep-live-cam/resolve/main/GFPGANv1.4.pth -P /content/deepfakecollab/Model

--2025-06-18 15:52:49--  https://huggingface.co/ninjawick/webui-faceswap-unlocked/resolve/main/inswapper_128.onnx
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.17, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/1d/30/1d30e44e7162b29b4d88bd541a73892ea95e967f8198019756868e03c15dc06c/e4a3f08c753cb72d04e10aa0f7dbe3deebbf39567d4ead6dce08e98aa49e16af?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27inswapper_128.onnx%3B+filename%3D%22inswapper_128.onnx%22%3B&Expires=1750265569&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDI2NTU2OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8xZC8zMC8xZDMwZTQ0ZTcxNjJiMjliNGQ4OGJkNTQxYTczODkyZWE5NWU5NjdmODE5ODAxOTc1Njg2OGUwM2MxNWRjMDZjL2U0YTNmMDhjNzUzY2I3MmQwNGUxMGFhMGY3ZGJlM2RlZWJiZjM5NTY3ZDRlYWQ2ZGNlMDhlOThhYTQ5ZTE2YWY%7EcmVzcG

### [Ignore]  DeBuggin (Run only when its necessery)



Debugging to ensure cuda was used. if not check the version of the cudnn, edit the install of the onnxruntime package to install the right version. https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements

In [6]:
!cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

#define CUDNN_MAJOR 9
#define CUDNN_MINOR 2
#define CUDNN_PATCHLEVEL 1
--
#define CUDNN_VERSION (CUDNN_MAJOR * 10000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */


In [7]:
import onnxruntime as rt
print(rt.__version__)

1.20.1


In [8]:
# Ensure the GPU providers are set explicitly
ort_session = rt.InferenceSession(
    "/content/deepfakecollab/Model/inswapper_128.onnx",
    providers=["TensorrtExecutionProvider", "CUDAExecutionProvider", "CPUExecutionProvider"]
)
# Verify the active provider again
print("Active providers:", ort_session.get_providers())

*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:507 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************
Active providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']


In [9]:
from onnxruntime import get_device
print("ONNX Runtime is using:", get_device())

ONNX Runtime is using: GPU


In [10]:
!nvidia-smi


Tue Jun 17 10:55:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P0             26W /   70W |    1140MiB /  15360MiB |     12%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Create the Colab Server



In your Google Colab notebook, set up a TCP server that will receive frames, process them using the FACE_SWAPPER model, and send back the results.

In [4]:
import socket
import cv2
import numpy as np
import insightface
import threading
import torch
import onnxruntime
from typing import Any
from insightface.app.common import Face
import matplotlib.pyplot as plt
# Check if CUDA is available
if 'CUDAExecutionProvider' in onnxruntime.get_available_providers():
    providers = ['CUDAExecutionProvider']
elif 'TensorrtExecutionProvider'  in onnxruntime.get_available_providers():
    providers = ['TensorrtExecutionProvider']
else:
    providers = ['CPUExecutionProvider']
print(providers)
FACE_SWAPPER = None
FACE_ANALYSER = insightface.app.FaceAnalysis(name='buffalo_l', providers=providers)
FACE_ANALYSER.prepare(ctx_id=0, det_size=(640, 640))
THREAD_LOCK = threading.Lock()
Frame = np.ndarray[Any, Any]
def get_face_swapper() -> Any:
    global FACE_SWAPPER
    with THREAD_LOCK:
        if FACE_SWAPPER is None:
            model_path = "/content/deepfakecollab/Model/inswapper_128.onnx"
            FACE_SWAPPER = insightface.model_zoo.get_model(model_path, providers=['CUDAExecutionProvider'])
    return FACE_SWAPPER
def swap_face(source_face: Face, target_face: Face, temp_frame: Frame) -> Frame:
    return get_face_swapper().get(temp_frame, target_face, source_face, paste_back=True)
def get_face_analyser() -> Any:
    #FACE_ANALYSER.prepare(ctx_id=0, det_size=(640, 640))
    return FACE_ANALYSER
def get_one_face(frame: Frame) -> Any:
    face = get_face_analyser().get(frame)
    try:
        return min(face, key=lambda x: x.bbox[0])
    except ValueError:
        return None
def get_many_faces(frame: Frame) -> Any:
    try:
        return get_face_analyser().get(frame)
    except IndexError:
        return None
def process_frame(source_face: Face, temp_frame: Frame,manyface: bool) -> Frame:
    if manyface:
        many_faces = get_many_faces(temp_frame)
        if many_faces:
            for target_face in many_faces:
                temp_frame = swap_face(source_face, target_face, temp_frame)
    else:
        target_face = get_one_face(temp_frame)
        if target_face:
            temp_frame = swap_face(source_face, target_face, temp_frame)
    return temp_frame
# Input and output ports for communication
# @title Streaming Ports
local_in_source = 5555 # @param {"type":"integer","placeholder":"5555"}
local_in_temp = 5556 # @param {"type":"integer","placeholder":"5556"}
local_out_frame = 5557 # @param {"type":"integer","placeholder":"5557"}

['CUDAExecutionProvider']
download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:02<00:00, 99338.84KB/s] 


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o



For Live Streaming from webcam run this cell but for just image swap run the next cell.



Note: Don't run both cells at same time. You should either run this or the one below




Set your region



Code | Region

--- | ---

us | United States

eu | Europe

ap | Asia/Pacific

au | Australia

sa | South America

jp | Japan

in | India

In [7]:
import zmq
import threading
import cv2
import numpy as np
import msgpack
import queue
import time
import zlib
from tqdm import tqdm
import subprocess
from collections import deque
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import deque
# Input and output ports for communication
# @title Streaming Ports
local_in_source = 5555 # @param {"type":"integer","placeholder":"5555"}
local_in_temp = 5556 # @param {"type":"integer","placeholder":"5556"}
local_out_frame = 5557 # @param {"type":"integer","placeholder":"5557"}
# @title [Optional] For Live Streaming
authtoken = "" # @param {"type":"string","placeholder":"ngrok token"}
region = "eu"# @param {"type":"string","placeholder":"eu"}
TimeFrate = 120 # @param {"type":"integer","placeholder":"120"}
GOP = "60" # @param {"type":"string","placeholder":"60"}
Bitrate = "3M" # @param {"type":"string","placeholder":"3M"}
Maxrate = "3M" # @param {"type":"string","placeholder":"3M"}
Bufsize = "5M" # @param {"type":"string","placeholder":"5M"}
# Initialize the executor with the number of workers
executor = ThreadPoolExecutor(max_workers=4)  # Adjust based on your CPU cores
# Batch size - tune this based on available resources and processing time per frame
BATCH_SIZE = 120
# Holds future results for parallel processing
future_to_batch = {}
#import matplotlib.pyplot as plt
def create_demo_image():
    # Create a demo image (e.g., a solid color or pattern)
    demo_image = np.zeros((540, 960, 3), dtype=np.uint8)  # Black image
    cv2.putText(demo_image, 'Demo Image', (50, 240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    return demo_image
def pull_socket(local_in_port):
    context = zmq.Context()
    socket = context.socket(zmq.REP)
    socket.setsockopt(zmq.RCVHWM, 100000)
    socket.setsockopt(zmq.LINGER, 0)
    address = f"tcp://127.0.0.1:{local_in_port}"
    socket.bind(address)  # Binding to a different local port
    print(f"PULL one socket bound to {address}")
    return socket
# Compress image
def compress_image(image, quality=95):
    # Set the JPEG quality parameter
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    # Encode the image as a JPEG
    result, encimg = cv2.imencode('.jpg', image, encode_param)
    if not result:
        raise Exception("Image encoding failed")
    # Decode the encoded image back to an image format
    decimg = cv2.imdecode(encimg, 1)
    return decimg
# Decompress image
def decompress_image(encimg):
    image = cv2.imdecode(np.frombuffer(encimg, np.uint8), cv2.IMREAD_COLOR)
    return image
# Global variables
frames_array=deque(maxlen=2000)
temp_frames_array=deque(maxlen=2000)
source_frame = None
is_manyFace = None
frameSize = '960x540'
fps = None
# Process a batch of frames simultaneously
# Function to process each frame
def process_frame_parallel(source, frame, many_face):
    return process_frame(get_one_face(source), frame, many_face)

def pull_worker(pull_socket):
    global source_frame,is_manyFace,frameSize,fps
    while True:
        try:
            # Receive the JSON with total chunks
            meta_data_json = pull_socket.recv_json()
            #print(meta_data_json)
            total_chunk = meta_data_json['total_chunk']
            # Send acknowledgment for metadata
            pull_socket.send_string("ACK")
            # Receive the array bytes
            source_array_bytes =b''
            for i in range(total_chunk):
                chunk = pull_socket.recv()
                source_array_bytes += chunk
                pull_socket.send_string(f"ACK {i + 1}/{total_chunk}")
            end_message = pull_socket.recv()
            if end_message == b"END":
                pull_socket.send_string("Final ACK")

            # Deserialize the bytes back to an ndarray
            source_array = np.frombuffer(source_array_bytes, dtype=np.dtype(meta_data_json['dtype_source'])).reshape(meta_data_json['shape_source'])
            source_frame = source_array
            is_manyFace = meta_data_json['manyface']
            frameSize =  meta_data_json['size']
            fps = meta_data_json["fps"]
                #process_queue.put(("source", source_array))
            break
        except zmq.Again:
            # Sleep briefly to avoid busy-waiting
            time.sleep(0.01)
        except Exception as e:
            print(f"Error: {e}")
def pull_worker_two(local_in_temp):
    ffmpeg_receive_command =  [
        'ffmpeg',
        '-i',f'tcp://127.0.0.1:{local_in_temp}?listen&fifo_size=100000&overrun_nonfatal=1',
        '-f','rawvideo',
        '-pix_fmt','bgr24',
        '-s','960x540',
        'pipe:1'
    ]
    ffmpeg_receive_process = subprocess.Popen((ffmpeg_receive_command), stdout=subprocess.PIPE)
    timefame =1/25
    print(f"InputStream ffmpeg bound from tcp://127.0.0.1:{local_in_temp}?listen")
    global source_frame,is_manyFace
    future_to_frame = {}
    while True:
        try:
            # Receive the JSON with total chunks
            # Read decoded frame from FFmpeg
            raw_frame = ffmpeg_receive_process.stdout.read(960 * 540 * 3)
            if not raw_frame:
                break
            framex = np.frombuffer(raw_frame, dtype=np.uint8).reshape((540, 960, 3))
            source_array = source_frame
            is_many_face = is_manyFace
            temp_frames_array.append(framex)
            if source_frame is not None:
                while len(temp_frames_array)>1:
                    t_frames = temp_frames_array[-1]
                    frames_array.append(t_frames)
                    temp_frames_array.popleft()
        except zmq.Again:
            # Sleep briefly to avoid busy-waiting
            time.sleep(0.01)
        except Exception as e:
            print(f"Error: {e}")
def push_worker(local_out_frame):
    source_array = None
    temp_array = None
    is_many_face = None
    timefrate = TimeFrate
    maxcount = 0
    global source_frame,is_manyFace,frameSize,fps
    print(f"OutputStream ffmpeg bound to tcp://127.0.0.1:{local_out_frame}?listen")
    ffmpeg_encode_command = [
        'ffmpeg',
        '-fflags','+nobuffer',
        '-flags', 'low_delay',
        '-probesize', '32',
        '-f', 'rawvideo',
        '-pix_fmt', 'bgr24',
        '-s', '960x540',
        '-r', str(timefrate),
        '-i', 'pipe:',
        '-c:v', 'libx264',  # Enable encoding for lower bandwidth usage
        '-preset', 'ultrafast',  # Fast encoding to reduce latency
        '-tune', 'zerolatency',  # Low latency setting
        '-g', GOP,#'60',  # GOP size set to 1 to make each frame a keyframe
        '-b:v', Bitrate,#'3000k',  # Set bitrate to 1 Mbps
        '-maxrate', Maxrate,#'3000k',
        '-bufsize', Bufsize,#'5000k',
        '-threads', '8',  # Enable threading
        '-f', 'mpegts',
        f'tcp://127.0.0.1:{local_out_frame}?listen&fifo_size=50000&overrun_nonfatal=1'
    ]
    timefame =1/timefrate
    ffmpeg_encode_process = subprocess.Popen((ffmpeg_encode_command), stdin=subprocess.PIPE)
    demo_image = create_demo_image()
    frames_to_skip = 200  # Number of frames to skip to reduce delay
    frame_count =0
    wait_frame = 0
    try:
        while True:
            # Get the processed array and metadata from the queue
            if len(frames_array)>wait_frame:
                if wait_frame<timefrate:
                    wait_frame += 1
                    #timefrate -=1

                temp_array = frames_array[-1]#frames_array.popleft()#process_queue.get()
                source_array = source_frame
                is_many_face = is_manyFace
                #framex = temp_array
                #print(source_array,temp_array)
                processed_array = process_frame(get_one_face(source_frame),temp_array,is_manyFace)
                framex = processed_array
                '''
                if source_frame is not None: #and temp_array is not None:
                    source_array = None
                    temp_array = None
                    is_many_face = None

                '''
            else:
                framex = demo_image
                # Write the frame to FFmpeg for encoding and streaming
            ffmpeg_encode_process.stdin.write(framex.tobytes())
            time.sleep(timefame)
    finally:
    #ffmpeg_receive_process.terminate()
        ffmpeg_encode_process.terminate()
# Create sockets
pull_socket = pull_socket(local_in_source)
# Run both workers in separate threads
# Start the pull worker thread
pull_thread = threading.Thread(target=pull_worker, args=(pull_socket,))
pull_thread.start()
# Start the push worker thread
pull_thread_two = threading.Thread(target=pull_worker_two, args=(local_in_temp,))
pull_thread_two.start()
# Start the push worker thread
push_thread = threading.Thread(target=push_worker,args=(local_out_frame,))
push_thread.start()

ZMQError: Address already in use



For image swapping run this cell.



Note: You can not run both cell at same time. Either This or The cell Above (For Live Streaming)

In [13]:
# @title [Optional] For Image Swap
import zmq
import threading
import cv2
import numpy as np
import msgpack
import queue
import time
import zlib
from tqdm import tqdm
import subprocess
from collections import deque
import matplotlib as plt
def push_socket(local_out_port):
    context = zmq.Context()
    socket = context.socket(zmq.REQ)
    socket.setsockopt(zmq.SNDHWM, 100000)
    socket.setsockopt(zmq.LINGER, 0)
    address = f"tcp://127.0.0.1:{local_out_port}"
    socket.bind(address)  # Binding to a local port
    print(f"PUSH socket bound to {address}")
    return socket
def pull_socket(local_in_port):
    context = zmq.Context()
    socket = context.socket(zmq.REP)
    socket.setsockopt(zmq.RCVHWM, 100000)
    socket.setsockopt(zmq.LINGER, 0)
    address = f"tcp://127.0.0.1:{local_in_port}"
    socket.bind(address)  # Binding to a different local port
    print(f"PULL one socket bound to {address}")
    return socket
# Compress image
def compress_image(image, quality=95):
    # Set the JPEG quality parameter
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    # Encode the image as a JPEG
    result, encimg = cv2.imencode('.jpg', image, encode_param)
    if not result:
        raise Exception("Image encoding failed")
    # Decode the encoded image back to an image format
    decimg = cv2.imdecode(encimg, 1)
    return decimg
# Decompress image
def decompress_image(encimg):
    image = cv2.imdecode(np.frombuffer(encimg, np.uint8), cv2.IMREAD_COLOR)
    return image
# Global variables
frames_array=deque(maxlen=2000)
source_frame = None
is_manyFace = None
frameSize = '640x480'
fps = None
#functions
def pull_worker(pull_socket):
    global source_frame,is_manyFace,frameSize,fps
    while True:
        try:
            # Receive the JSON with total chunks
            meta_data_json = pull_socket.recv_json()
            #print(meta_data_json)
            total_chunk = meta_data_json['total_chunk']
            # Send acknowledgment for metadata
            pull_socket.send_string("ACK")
            # Receive the array bytes
            source_array_bytes =b''
            for i in range(total_chunk):
                chunk = pull_socket.recv()
                source_array_bytes += chunk
                pull_socket.send_string(f"ACK {i + 1}/{total_chunk}")
            end_message = pull_socket.recv()
            if end_message == b"END":
                pull_socket.send_string("Final ACK")
            # Deserialize the bytes back to an ndarray
            source_array = np.frombuffer(source_array_bytes, dtype=np.dtype(meta_data_json['dtype_source'])).reshape(meta_data_json['shape_source'])
            #plt.imshow(source_array[:, :, ::-1])
            #plt.show()
            #frame_queue.append(["source", source_array])
            source_frame = source_array
            is_manyFace = meta_data_json['manyface']
            frames_array.append(["source",source_array,is_manyFace])
                #process_queue.put(("source", source_array))
            #break
        except zmq.Again:
            # Sleep briefly to avoid busy-waiting
            time.sleep(0.01)
        except Exception as e:
            print(f"Error: {e}")
def pull_worker_two(pull_socket_two):
    while True:
        try:
           # Receive the JSON with total chunks
            meta_data_json = pull_socket_two.recv_json()
            #print(meta_data_json)
            total_chunk = meta_data_json['total_chunk']
            # Send acknowledgment for metadata
            pull_socket_two.send_string("ACK")
            # Receive the array bytes
            temp_array_bytes =b''
            for i in range(total_chunk):
                chunk = pull_socket_two.recv()
                temp_array_bytes += chunk
                pull_socket_two.send_string(f"ACK {i + 1}/{total_chunk}")
            end_message = pull_socket_two.recv()
            if end_message == b"END":
                pull_socket_two.send_string("Final ACK")
            # Deserialize the bytes back to an ndarray
            temp_array = np.frombuffer(temp_array_bytes, dtype=np.dtype(meta_data_json['dtype_temp'])).reshape(meta_data_json['shape_temp'])
            #if source_frame is not None:
            frames_array.append(["temp",temp_array])
            print("added",len(frames_array))
            #break
        except zmq.Again:
            # Sleep briefly to avoid busy-waiting
            time.sleep(0.01)
        except Exception as e:
            print(f"Error: {e}")
def push_worker(push_socket):
    global source_frame,is_manyFace,frameSize,fps
    temp_frm = None
    source_frm = None
    is_manyface = None
    try:
        while True:
            # Get the processed array and metadata from the queue
            if len(frames_array)>0:
                #print(frames_array)
                item = frames_array.popleft()#process_queue.get()
                if item[0]=="source":
                    source_frm = item[1]
                    is_manyface = item[2]
                if item[0]=="temp":
                    temp_frm = item[1]
                    print("Recieved")
                if temp_frm is not None and source_frm is not None:
                    processed_frm =process_frame(get_one_face(source_frm),temp_frm,is_manyface)
                    face_bytes = processed_frm.tobytes()
                    chunk_size = 1024*200
                    total_chunk = len(face_bytes) // chunk_size + 1
                    metadata ={
                        'dtype_source':str(processed_frm.dtype),
                        'shape_source':processed_frm.shape,
                        'size':'640x480',
                        'fps':'60'
                        #'shape_temp':temp_frame.shape
                    }
                    new_metadata = {'total_chunk': total_chunk}
                    metadata.update(new_metadata)
                    # Send metadata first
                    push_socket.send_json(metadata)
                    # Wait for acknowledgment for metadata
                    ack = push_socket.recv_string()
                    with tqdm(total=total_chunk, desc="Sending chunks", unit="chunk") as pbar:
                        for i in range(total_chunk):
                            chunk = face_bytes[i * chunk_size:(i + 1) * chunk_size]
                            # Send the chunk
                            push_socket.send(chunk)
                            # Wait for acknowledgment after sending each chunk
                            ack = push_socket.recv_string()
                            pbar.set_postfix_str(f'Chunk {i + 1}/{total_chunk} ack: {ack}')
                            pbar.update(1)
                    # Send a final message to indicate all chunks are sent
                    push_socket.send(b"END")
                    # Wait for the final reply
                    final_reply_message = push_socket.recv_string()
                    print(f"Received final reply: {final_reply_message}")
                    temp_frm = None
                    source_frm = None
                    is_manyface = None
    except Exception as e:
        print (f"Error in Push Wokr {e}")
local_in_source = 5555 # @param {"type":"integer","placeholder":"5555"}
local_in_temp = 5556 # @param {"type":"integer","placeholder":"5556"}
local_out_frame = 5557 # @param {"type":"integer","placeholder":"5557"}
# Create sockets
pull_socket_ = pull_socket(local_in_source)
pull_socket_two = pull_socket(local_in_temp)
push_socket_ = push_socket(local_out_frame)
# Run both workers in separate threads
# Start the pull worker thread
pull_thread = threading.Thread(target=pull_worker, args=(pull_socket_,))
pull_thread.start()
# Start the push worker thread
pull_thread_two = threading.Thread(target=pull_worker_two, args=(pull_socket_two,))
pull_thread_two.start()
# Start the push worker thread
push_thread = threading.Thread(target=push_worker,args=(push_socket_,))
push_thread.start()

ZMQError: Address already in use

In [ ]:
# @title [Optional] Open FRP tunnel
HostIP = "194.113.64.71" # @param {"type":"string","placeholder":"0.0.0.0"}
frpc_config = f"""serverAddr = "{HostIP}"
serverPort = 7000
[[proxies]]
name = "Pull-tcp"
type = "tcp"
localIP = "127.0.0.1"
localPort = {local_in_source}
remotePort = 6000
[[proxies]]
name = "Pull-tcp_two"
type = "tcp"
localIP = "127.0.0.1"
localPort = {local_in_temp}
remotePort = 6001
[[proxies]]
name = "Push-tcp"
type = "tcp"
localIP = "127.0.0.1"
localPort = {local_out_frame}
remotePort = 6002"""
with open('frp_0.59.0_linux_amd64/frpc.toml', 'w') as f:
    f.write(frpc_config)
#Remote tcp
print(f'tcp://{str(HostIP)}:6000 ---> {local_in_source}')
print(f'tcp://{str(HostIP)}:6001 ---> {local_in_temp}')
print(f'tcp://1{str(HostIP)}:6002 ---> {local_out_frame}')

from subprocess import Popen, PIPE
import time
ps = Popen('/kaggle/working/deepfakecollab/Scripts/open_tunnel_frs.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)
!/kaggle/working/deepfakecollab/Scripts/open_tunnel_frs.sh

In [ ]:
# @title [Optional] Open Ngrok tunnel
from subprocess import Popen, PIPE
import shlex
import json
import time
def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()
def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info
  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'inputtwo':
      in_addrtwo = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')
  return in_addr,in_addrtwo, out_addr
config =\
f"""
version: 2
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_source}
    proto: tcp
  inputtwo:
    addr: {local_in_temp}
    proto: tcp
  output:
    addr: {local_out_frame}
    proto: tcp
"""
with open('ngrok.conf', 'w') as f:
  f.write(config)
# (Re)Open tunnel
ps = Popen('/content/deepfakecollab/Scripts/open_tunnel_ngrok.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)
# Get tunnel addresses
try:
  in_addr,in_addr_two, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")

##  **KILL PROCESS (PORTS)**

In [ ]:
import subprocess
# List of ports to free
ports = [5555, 5556, 5557]
for port in ports:
    # Find the process ID (PID) using the port
    result = subprocess.run(f"lsof -t -i:{port}", shell=True, capture_output=True, text=True)
    pid = result.stdout.strip()

    # Kill the process if it exists
    if pid:
        subprocess.run(f"kill -9 {pid}", shell=True)
        print(f"Terminated process on port {port} with PID {pid}")
    else:
        print(f"No process found on port {port}")
